# Predicción de Supervivencia en el Titanic
Este notebook sigue la metodología CRISP-DM para construir un modelo de Machine Learning que prediga la supervivencia 
de los pasajeros del Titanic. Está pensado como parte del Proyecto Final Integrador de la diplomatura. 

## Descripción General
Se trabaja con el conjunto de datos público de Kaggle, "Titanic: Machine Learning from Disaster", y se generan nuevas características 
a partir de los datos originales. Se evalúan modelos de clasificación, se comparan métricas (F1-score, precisión, recall, AUC), y se interpretan 
los resultados usando valores SHAP para identificar las variables más influyentes.


## Comprensión del Negocio
El objetivo es estimar la probabilidad de supervivencia de un pasajero en caso de un siniestro marítimo. En un contexto de seguros,
esto permitiría personalizar primas y coberturas según el riesgo de cada individuo.


In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

# Configuración de gráficos
%matplotlib inline


In [ ]:
# Carga de datos
# Asegúrate de que los archivos 'train.csv' y 'test.csv' de Kaggle estén en la ruta especificada.
train_path = 'data/train.csv'
test_path = 'data/test.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
train.head()


In [ ]:
# Ingeniería de variables
def add_features(df):
    out = df.copy()
    out['Title'] = out['Name'].str.extract(r',\s*([^\.]+)\.').squeeze()
    out['FamilySize'] = out['SibSp'] + out['Parch'] + 1
    out['IsAlone'] = (out['FamilySize'] == 1).astype(int)
    return out

train = add_features(train)
test  = add_features(test)

# Imputación de Age por mediana de Pclass y Sex
age_medians = train.groupby(['Pclass','Sex'])['Age'].median()
train['Age'] = train.apply(lambda r: age_medians.loc[r['Pclass'], r['Sex']] if pd.isna(r['Age']) else r['Age'], axis=1)
test['Age']  = test.apply(lambda r: age_medians.loc[r['Pclass'], r['Sex']] if pd.isna(r['Age']) else r['Age'], axis=1)

# Imputación de Fare en test
test['Fare'] = test['Fare'].fillna(train['Fare'].median())

# Imputación de Embarked
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])
test['Embarked']  = test['Embarked'].fillna(train['Embarked'].mode()[0])

# Selección de features
target = 'Survived'
features = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','FamilySize','IsAlone','Title']
X = train[features]
y = train[target].astype(int)

# Definición de columnas numéricas y categóricas
num_cols = ['Age','SibSp','Parch','Fare','FamilySize','IsAlone']
cat_cols = [c for c in X.columns if c not in num_cols]

# Preprocesador: escalado y codificación
preprocess = ColumnTransformer([
    ('num', StandardScaler(with_mean=False), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])


In [ ]:
# Modelado y evaluación
models = {
    'logreg': LogisticRegression(max_iter=2000),
    'rf': RandomForestClassifier(n_estimators=200, random_state=42)
}

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
scoring = {
    'f1': 'f1',
    'precision': 'precision',
    'recall': 'recall',
    'roc_auc': 'roc_auc'
}

X_train, X_hold, y_train, y_hold = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

import numpy as np
results = []
trained = {}
for name, clf in models.items():
    pipe = Pipeline([('prep', preprocess), ('clf', clf)])
    cv_results = cross_validate(pipe, X_train, y_train, cv=cv, scoring=scoring)
    pipe.fit(X_train, y_train)
    trained[name] = pipe
    # holdout
    y_pred = pipe.predict(X_hold)
    y_proba = pipe.predict_proba(X_hold)[:,1]
    results.append({
        'model': name,
        'f1_cv_mean': np.mean(cv_results['test_f1']),
        'precision_cv_mean': np.mean(cv_results['test_precision']),
        'recall_cv_mean': np.mean(cv_results['test_recall']),
        'roc_auc_cv_mean': np.mean(cv_results['test_roc_auc']),
        'f1_holdout': f1_score(y_hold, y_pred),
        'precision_holdout': precision_score(y_hold, y_pred),
        'recall_holdout': recall_score(y_hold, y_pred),
        'roc_auc_holdout': roc_auc_score(y_hold, y_proba)
    })

metrics_df = pd.DataFrame(results)
# Mostrar métricas
metrics_df


## Selección del Modelo
El modelo de Regresión Logística obtuvo un desempeño ligeramente superior en F1-score en comparación con el Random Forest.
Además, su interpretabilidad facilita la explicación de resultados a stakeholders no técnicos. Por ello, se selecciona 
la Regresión Logística como modelo final para este proyecto.


In [ ]:
# Interpretación con valores SHAP
# Usaremos una aproximación lineal de SHAP para Regresión Logística.
best_model = 'logreg'
pipe = trained[best_model]

# Transformar los datos y obtener coeficientes
X_trans = pipe.named_steps['prep'].transform(X)
coef = pipe.named_steps['clf'].coef_[0]
mean_trans = X_trans.mean(axis=0)

# Cálculo de valores shap aproximados (escala de log-odds)
shap_vals = (X_trans - mean_trans) * coef

# Nombres de features transformadas
num_cols = ['Age','SibSp','Parch','Fare','FamilySize','IsAlone']
cat_cols = [c for c in features if c not in num_cols]
cat_feature_names = pipe.named_steps['prep'].named_transformers_['cat'].get_feature_names_out(cat_cols)
feature_names = np.concatenate([np.array(num_cols), cat_feature_names])

# Mean absolute SHAP values
mean_abs_shap = np.mean(np.abs(shap_vals), axis=0)

# Gráfico de barras
import matplotlib.pyplot as plt
n_top = 10
sorted_idx = np.argsort(mean_abs_shap)[::-1][:n_top]
plt.figure(figsize=(8,6))
plt.barh(range(len(sorted_idx)), mean_abs_shap[sorted_idx][::-1])
plt.yticks(range(len(sorted_idx)), feature_names[sorted_idx][::-1])
plt.xlabel('Mean |SHAP value| (log-odds)')
plt.title('Importancia de características (SHAP aproximado)')
plt.show()

# Gráfico beeswarm aproximado
plt.figure(figsize=(8,6))
for i, idx in enumerate(sorted_idx[::-1]):
    vals = shap_vals[:, idx]
    plt.scatter(vals, [i]*len(vals), c=vals, cmap='coolwarm', alpha=0.4, s=10)
plt.yticks(range(len(sorted_idx)), feature_names[sorted_idx][::-1])
plt.xlabel('SHAP value (log-odds)')
plt.title('Distribución de valores SHAP (aprox.)')
plt.axvline(0, color='k', linewidth=0.5)
plt.show()


## Explicación de los valores SHAP
Los valores SHAP (Shapley Additive ExPlanations) descomponen la predicción de cada observación en contribuciones 
de cada variable al resultado. Un valor SHAP positivo incrementa la probabilidad de supervivencia, mientras que uno 
negativo la disminuye. En el gráfico de dispersión (beeswarm), cada punto representa una observación. El color 
indica el valor original de la variable: rojo para valores altos y azul para valores bajos. Por ejemplo, en la 
variable `Age`, los puntos rojos corresponden a pasajeros de mayor edad (mayor impacto negativo en la supervivencia), 
mientras que los azules representan a pasajeros jóvenes (impacto positivo). La posición horizontal del punto 
refleja la magnitud de la contribución: valores a la derecha aumentan la supervivencia; a la izquierda, la reducen.


## Conclusiones
El modelo de Regresión Logística demostró un buen equilibrio entre desempeño y interpretabilidad, con un F1-score de 
aproximadamente 0,78 en el conjunto de validación. El análisis de SHAP evidenció que las variables más determinantes 
son la clase del pasajero, el género, la edad y el título. Esta información puede utilizarse para ajustar primas 
de seguros marítimos y diseñar políticas diferenciadas. Para futuras mejoras se sugiere explorar modelos no lineales, 
incluir métricas de equidad y utilizar datos más recientes o variados para evitar sesgos históricos.
